## <center> RecSys. Home Assignment 1</center>

Один из важных навыков для построения рекомендательных систем - это умение корректно считать метрики качества ранжирования.

В этой домашке мы предлагаем вам потренироваться в этом, и имплементировать метрики Precision@k, Recall@k, MNAP@k и NDCG@k по формулам, чтобы дальше переиспользовать при построении рекомендательных моделей. 

Критерии оценивания:
* Что-то пытался сделать, дописал свой код, но ничего не получилось - 1 балл. 
* Не совсем корректная имплементация одной из 4 метрик, прохождение части тестов - 1 балл. 
* Корректная имплементация одной из 4 метрик, прохождение всех тестов - 2 балла. 
* +1 балл, если получится написать Precision@k, Recall@k без циклов.
* +1 балл, если получится написать NDCG@k, MNAP@k без циклов.

Дедлайн сдачи - **10 октября 23:59**. 

Формат сдачи - отправить Jupyter notebook на почту ananyeva.me@gmail.com с темой письма "[RecSys HW1]" и названием файла Name_Surname_HW1.ipynb.  

Удачи!

In [1]:
from typing import NamedTuple, Union
import tests
import torch

In [2]:
class PrepareTargetResult(NamedTuple):
    values: torch.Tensor
    indices: torch.Tensor


def validate_metric_inputs(output: torch.Tensor, target: torch.Tensor) -> None:
    if output.size() != target.size():
        raise IndexError(
            "Unequal sizes for output and target: "
            f"output - {output.size()}, target - {target.size()}."
        )
    if not (target.eq(0) | target.eq(1)).all():
        raise ValueError(
            "Target contains values outside of 0 and 1." f"\nTarget:\n{target}"
        )


def prepare_target(
    output: torch.Tensor, target: torch.Tensor, return_indices: bool = False
) -> Union[torch.Tensor, PrepareTargetResult]:
    validate_metric_inputs(output, target)
    # Define order by sorted output scores.
    indices = output.argsort(dim=-1, descending=True)
    sorted_target = torch.gather(target, index=indices, dim=-1)
    return (
        PrepareTargetResult(sorted_target, indices) if return_indices else sorted_target
    )


def nan_to_num(tensor: torch.Tensor, nan: float = 0.0) -> torch.Tensor:
    return torch.where(
        torch.isnan(tensor) | torch.isinf(tensor),
        torch.full_like(tensor, fill_value=nan),
        tensor,
    )

# Precision

$$ P\@k = \frac{\sum_{i=1}^k [rel_{i}]}{k} $$

In [3]:
def precision(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    # output, target ~ (users, items)
    # target_sorted_by_output ~ (users, items)
    target_sorted_by_output = prepare_target(output, target)
    # YOUR CODE HERE
    topk = min(topk, target_sorted_by_output.shape[1])
    return (target_sorted_by_output[:, :topk].sum(1) / topk).mean()

In [4]:
tests.run_precision(precision)

# Recall

$$P\@k = \frac{\sum_{i=1}^k [rel_{i}]}{|rel_k|}$$

In [5]:
def recall(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    # output, target ~ (users, items)
    # target_sorted_by_output ~ (users, items)
    target_sorted_by_output = prepare_target(output, target)
    # YOUR CODE HERE
    topk = min(topk, target_sorted_by_output.shape[1])
    nrel = target_sorted_by_output.sum(1)
    res = torch.zeros((target_sorted_by_output.shape[0],))
    res[nrel != 0] = target_sorted_by_output[:, :topk].sum(1) / nrel
    return res.mean()

In [6]:
tests.run_recall(recall)

# Mean (Normalized) Average Precision


$$AP\@k = \frac{\sum_{i=1}^{k} P\@i \cdot [rel\@ i]} {\sum_{i=1}^{k}[rel\@i]}$$
$$MNAP\@k = \frac{1}{N} \sum_{k=1}^{N} \frac{1}{min(k, m_u)} AP\@k,$$

где $m_u$ - количество items с интеракциями у пользователя $u$ в тестовый период, <br>
$N$ - количество пользователей. 

In [7]:
def mnap(output: torch.Tensor, target: torch.Tensor, topk: int, normalized: bool = True) -> torch.Tensor:
    # output, target ~ (users, items)
    # target_sorted_by_output ~ (users, items)
    target_sorted_by_output, indices = prepare_target(output, target, return_indices=True)
    sorted_output = torch.gather(output, index=indices, dim=-1)
    # YOUR CODE HERE
    topk = min(topk, target_sorted_by_output.shape[1])
    target_sorted_by_output = target_sorted_by_output[:, :topk]
    p_i = torch.zeros_like(target_sorted_by_output)
    for k in range(1, topk + 1):
        p_i[:, k-1] = (target_sorted_by_output[:, :k].sum(1) / k) * target_sorted_by_output[:, k-1]

    ap_k = p_i.sum(1) / (p_i != 0).sum(1)
    ap_k[torch.isnan(ap_k)] = 0

    if normalized:
        m_u = (sorted_output != 0).sum(1)
        k = torch.full_like(m_u, topk)
        return (ap_k / torch.where(k < m_u, k, m_u)).mean()

    return ap_k.mean()

In [8]:
tests.run_map(mnap)

AssertionError: Inputs:{
  "output": [
    [
      0.5,
      0.4000000059604645,
      0.30000001192092896,
      0.20000000298023224
    ]
  ],
  "target": [
    [
      1.0,
      0.0,
      1.0,
      0.0
    ]
  ],
  "topk": 3
}
namespace(number_of_elements=1, total_mismatches=1, max_abs_diff=0.2777777910232544, max_abs_diff_idx=0, atol=1e-05, max_rel_diff=0.5, max_rel_diff_idx=0, rtol=1.3e-06)

In [9]:
from catalyst.metrics import mean_average_precision

def catalyst_map(output, target, topk):
    return mean_average_precision(outputs=output, targets=target, topk=topk)

tests.run_catalyst_map(catalyst_map)

/Users/germanarutunov/Projects/ftiad/venv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


AssertionError: Inputs:{
  "output": [
    [
      0.5,
      0.4000000059604645,
      0.30000001192092896,
      0.20000000298023224
    ]
  ],
  "target": [
    [
      1.0,
      0.0,
      1.0,
      0.0
    ]
  ],
  "topk": 3
}
namespace(number_of_elements=1, total_mismatches=1, max_abs_diff=0.2777777910232544, max_abs_diff_idx=0, atol=1e-05, max_rel_diff=0.5, max_rel_diff_idx=0, rtol=1.3e-06)

In [10]:
tests.run_mnap(mnap)

AssertionError: Inputs:{
  "output": [
    [
      0.0,
      0.0,
      0.0,
      0.0
    ]
  ],
  "target": [
    [
      0.0,
      0.0,
      0.0,
      0.0
    ]
  ],
  "topk": 1
}
namespace(number_of_elements=1, total_mismatches=1, max_abs_diff=nan, max_abs_diff_idx=0, atol=1e-05, max_rel_diff=nan, max_rel_diff_idx=0, rtol=1.3e-06)

# Normalized Dicsounted Cumulative Gain


$$ NDCG \@k = \frac{DCG\@k}{IDCG\@k},$$ где
$$DCG\@k = \sum_{i=1}^{k} \frac{2^{rel_{i}} - 1}{log_2 (i + 1)}$$
$$IDCG\@k = \sum_{i=1}^{|rel_{k}|} \frac{2^{rel_{i}} - 1}{log_2 (i + 1)}$$

In [11]:
def dcg(tensor: torch.Tensor) -> torch.Tensor:
    gains = (2**tensor) - 1
    return gains / torch.log2(torch.arange(0, tensor.size(-1), dtype=torch.float, device=tensor.device) + 2.0)


def ndcg(output: torch.Tensor, target: torch.Tensor, topk: int) -> torch.Tensor:
    # output, target ~ (users, items)
    # target_sorted_by_output ~ (users, items)
    target_sorted_by_output = prepare_target(output, target)
    sorted_target = prepare_target(target, target)
    a = dcg(target_sorted_by_output[:, :topk]).sum(1)
    b = dcg(sorted_target[:, :topk]).sum(1)
    rel = b != 0
    a[~rel] = 0
    a[rel] /= b[rel]

    return a.mean()

In [12]:
tests.run_ndcg(ndcg)

AssertionError: Inputs:{
  "output": [
    [
      9.0,
      5.0,
      3.0,
      0.0,
      7.0,
      4.0,
      0.0,
      0.0,
      6.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      1.0,
      8.0,
      2.0,
      0.0,
      10.0
    ],
    [
      0.0,
      0.0,
      1.0,
      5.0,
      9.0,
      3.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      4.0,
      0.0,
      0.0,
      10.0,
      7.0,
      0.0,
      2.0,
      8.0,
      6.0
    ],
    [
      0.0,
      1.0,
      4.0,
      8.0,
      6.0,
      5.0,
      3.0,
      7.0,
      10.0,
      0.0,
      9.0,
      0.0,
      0.0,
      2.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0
    ],
    [
      7.0,
      8.0,
      0.0,
      0.0,
      1.0,
      0.0,
      4.0,
      0.0,
      10.0,
      0.0,
      0.0,
      6.0,
      0.0,
      0.0,
      0.0,
      9.0,
      2.0,
      3.0,
      5.0,
      0.0
    ]
  ],
  "target": [
    [
     

In [13]:
from catalyst.metrics import ndcg as cndcg

def catalyst_ndcg(output, target, topk):
    return cndcg(output, target, topk)

tests.run_catalyst_ndcg(catalyst_ndcg)

AssertionError: Inputs:{
  "output": [
    [
      9.0,
      5.0,
      3.0,
      0.0,
      7.0,
      4.0,
      0.0,
      0.0,
      6.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      1.0,
      8.0,
      2.0,
      0.0,
      10.0
    ],
    [
      0.0,
      0.0,
      1.0,
      5.0,
      9.0,
      3.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      4.0,
      0.0,
      0.0,
      10.0,
      7.0,
      0.0,
      2.0,
      8.0,
      6.0
    ],
    [
      0.0,
      1.0,
      4.0,
      8.0,
      6.0,
      5.0,
      3.0,
      7.0,
      10.0,
      0.0,
      9.0,
      0.0,
      0.0,
      2.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0,
      0.0
    ],
    [
      7.0,
      8.0,
      0.0,
      0.0,
      1.0,
      0.0,
      4.0,
      0.0,
      10.0,
      0.0,
      0.0,
      6.0,
      0.0,
      0.0,
      0.0,
      9.0,
      2.0,
      3.0,
      5.0,
      0.0
    ]
  ],
  "target": [
    [
     